### Compute and store filtered and demodulated velocity fields at each grid point

In [1]:
import numpy as np
import geopandas as gpd

import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from scipy import signal
import scipy.ndimage as im

from sympy import Symbol, pi, atan, factor, lambdify


import mitequinox.plot as pl
import mitequinox.sigp as sp

from fsspec.implementations.local import LocalFileSystem

In [2]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
cluster = PBSCluster(processes=1, cores=1,walltime='04:00:00') #processes=7, cores=7
w = cluster.scale(jobs=10
                 )
client = Client(cluster)

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
client

<Client: 'tcp://10.148.1.77:50326' processes=10 threads=10, memory=1.09 TiB>

In [4]:
def convolve(x, h=None, hilbert=False):
    """ Convolve an input signal with a kernel
    Optionaly compute the Hilbert transform of the resulting time series
    
    Parameters
    x : input signal
    h : filter 
    hilbert : True for Hilbert transform to be applied to the filtered signal
    
    Returns
    x_f : filtered signal or hilbert transform of the filtered signal
    """
    x_f = signal.filtfilt(h, [1], x, axis=-1,padlen=0)#
    if hilbert:
        return signal.hilbert(x_f)
    else:
        return x_f

def filt(v, h, hilbert=False):
    
    output_dtype = complex if hilbert else float
    gufunc_kwargs = dict(output_sizes={'time': len(v.time)})
    return xr.apply_ufunc(convolve, v, kwargs={'h': h, 'hilbert': hilbert},
                    dask='parallelized', output_dtypes=[output_dtype],
                    input_core_dims=[['time']],
                    output_core_dims=[['time']],
                    dask_gufunc_kwargs = gufunc_kwargs,
                         )

##### First step : Filter and demodulate for each face

In [5]:
# Load Eulerian fields
ds =  xr.open_zarr(ut.work_data_dir+'rechunked/SSU_rot.zarr',consolidated=False)
grd = ut.load_grd(consolidated=False)[['XC', 'YC', 'Depth']]#.persist()

In [5]:
#isel=dict(face=12) #face
#V = ['SSV_rot']#'zonal_velocity','meridional_velocity'
path = '/home1/datawork/zcaspar/mit4320/filtered_itide/'
dsel = 200

In [6]:
dt = 1/24 # time step in days

tidal_omega = sp.get_tidal_frequencies("M2", "K2","S2","N2")
omega_M2,omega_S2,omega_N2,omega_K2, domega, name = tidal_omega["M2"],tidal_omega["S2"],tidal_omega["N2"],tidal_omega["K2"], .2, "semidiurnal"
omega = (omega_M2+omega_S2)/2#center frequency
Tw = 30 #filter length
dband = 0.2 # half-bandwidth
V = ['SSU_rot']

In [7]:
#dt = 1. # in hours
#T = 20
#omega = 1/30
#h = signal.firwin(T*24, cutoff=[omega], pass_zero=True, nyq=1./2/dt, scale=True)
#V = 'SSU_rot'

In [8]:
import dask
def wrap_filter(_ds,dt=dt):
    h = sp.generate_filter(om, T=Tw, dt=dt, bandwidth=dband, normalized_bandwidth=None)
    time = np.arange(0,_ds['time'].values.size*dt,dt)
    exp = np.exp(-1j*om*2*np.pi*time)
#    _ds = ut._reset_chunk_encoding(_ds)
    _ds = _ds.chunk(dict(time=-1,i=100,j=100))
    ds_hat={}
    for v in V:
        ds_hat[v+'_hat'] = filt(_ds[v], h,hilbert=False)#.persist()
        ds_hat[v+'_hat_real'] = ds_hat[v+'_hat'].real
        ds_hat[v+'_hat_imag'] = ds_hat[v+'_hat'].imag
#        ds_hat[v+'_hat'] = ds_hat[v+'_hat'].assign_coords({'lon':ds_hat[v+'_hat'].lon,'lat':ds_hat[v+'_hat'].lat})
#        ds_hat[v+'_demodulated'] = ds_hat[v+'_hat']*exp
#        ds_hat[v+'_demodulated_real'] = ds_hat[v+'_demodulated'].real
#        ds_hat[v+'_demodulated_imag'] = ds_hat[v+'_demodulated'].imag
    ds_hat = xr.merge([ds_hat[v].rename(v) for v in ds_hat.keys()])#.drop([v+'_hat',v+'_demodulated'])
#    ds_hat = ds_hat.chunk(dict(i=100)) #,j=4320//4
    ds_hat = ut._reset_chunk_encoding(ds_hat)
    return ds_hat#.drop(['XC','YC'])

def wrap_filter_low(_ds,dt=dt):
    h = signal.firwin(T*24, cutoff=[omega], pass_zero=True, nyq=1./2/dt, scale=True)
    ds_hat = filt(_ds[v], h,hilbert=False)#.persist()
    ds_hat = ds_hat.rename(v+'_hat').to_dataset()
#    ds_hat = ds_hat.chunk(dict(i=4320//4,j=4320//4))
#    ds_hat = ut._reset_chunk_encoding(ds_hat)
    return ds_hat

In [9]:
def wrap_demodulation(ds_hat,dt=dt):
#    time = np.arange(0,ds_hat['time'].values.size*dt,dt)
    exp = np.exp(-1j*om*2*np.pi*time)

#    _ds = ut._reset_chunk_encoding(_ds)
#    _ds = _ds.chunk(dict(time=16,i=100,j=100))
#    ds_hat={}
    for v in V:
#        ds_hat[v+'_hat'] = ds_hat[v+'_hat_real']+1j*ds_hat[v+'_hat_imag']
#        ds_hat[v+'_hat'] = ds_hat[v+'_hat'].assign_coords({'lon':ds_hat[v+'_hat'].lon,'lat':ds_hat[v+'_hat'].lat})
        ds_hat[v+'_demodulated'] = ds_hat[v+'_hat']*exp
        ds_hat[v+'_demodulated_real'] = ds_hat[v+'_demodulated'].real
        ds_hat[v+'_demodulated_imag'] = ds_hat[v+'_demodulated'].imag
    ds_hat = xr.merge([ds_hat[v].rename(v) for v in ds_hat.keys()]).drop([v+'_demodulated',v+'_hat'])
 #   ds_hat = ds_hat.chunk(dict(i=100)) #,j=4320//4
    
    return ds_hat#.drop(['XC','YC'])

In [11]:
#Select and rechunk (empirical) original dataset
isel = dict(j=slice(3800,None))#j=slice(0,4320//10))
dsp = ds.isel(**isel)#.isel(i=slice(4,None,4),j=slice(4, None,4))#.isel(face=0)#.sel(**sel)#.persist()
dsp = dsp.chunk({"time": -1,'i':100})#.persist()#,'i':100,'j':70
dsp

,Array,Chunk
Bytes,111.40 MiB,203.12 kiB
Shape,"(13, 520, 4320)","(1, 520, 100)"
Count,1444 Tasks,572 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,111.40 MiB,203.12 kiB
Shape,"(13, 520, 4320)","(1, 520, 100)"
Count,1444 Tasks,572 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [10]:
dsp =  xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_3800.zarr',consolidated=False)#.isel(face=slice(1,None))
dsp

,Array,Chunk
Bytes,1.84 TiB,659.18 MiB
Shape,"(13, 520, 4320, 8640)","(1, 100, 100, 8640)"
Count,3433 Tasks,3432 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,1.84 TiB,659.18 MiB
Shape,"(13, 520, 4320, 8640)","(1, 100, 100, 8640)"
Count,3433 Tasks,3432 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [11]:
time = np.arange(0,dsp['time'].values.size*dt,dt)

In [12]:
dsp#.isel(face=0)[V[0]+'_hat']

,Array,Chunk
Bytes,1.84 TiB,659.18 MiB
Shape,"(13, 520, 4320, 8640)","(1, 100, 100, 8640)"
Count,3433 Tasks,3432 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,1.84 TiB,659.18 MiB
Shape,"(13, 520, 4320, 8640)","(1, 100, 100, 8640)"
Count,3433 Tasks,3432 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [13]:
om,Tw,dband,V = omega,Tw,dband,V #define parameters needed in wrap_filter

In [14]:
# Apply wrap_filter on the dataset and store the result for each face (seperated)
import os
#zarr_main = os.path.join(ut.root_data_dir, "filtered_itide/SSU_filtered_face4.zarr")
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_out, zarr = ut.custom_distribute(dsp, 
                                        wrap_demodulation,
                                        overwrite=True,
                                        suffix="SSU_demodulated_3800.zarr",
                                        tmp_dir=os.path.join(ut.root_data_dir, "filtered_itide"),
                                        append=True,
                                        face=1
#                                        j=200
                                       )

13it [44:32, 205.54s/it]


In [38]:
ds_out

,Array,Chunk
Bytes,2.35 TiB,26.37 MiB
Shape,"(4320, 4320, 8640)","(100, 1080, 16)"
Count,95041 Tasks,95040 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,2.35 TiB,26.37 MiB
Shape,"(4320, 4320, 8640)","(100, 1080, 16)"
Count,95041 Tasks,95040 Chunks
Type,complex128,numpy.ndarray


In [13]:
ds_out

,Array,Chunk
Bytes,2.82 TiB,659.18 MiB
Shape,"(13, 800, 4320, 8640)","(1, 100, 100, 8640)"
Count,4577 Tasks,4576 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,2.82 TiB,659.18 MiB
Shape,"(13, 800, 4320, 8640)","(1, 100, 100, 8640)"
Count,4577 Tasks,4576 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [15]:
#client.restart()
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


##### Second step : Concat filtered and demodulated fields

In [2]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()

#
from dask_jobqueue import PBSCluster
cluster = PBSCluster(processes=7, cores=7, walltime='04:00:00') #processes=7, cores=7 ,walltime='03:00:00'
w = cluster.scale(jobs=8
                 )
client = Client(cluster)

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.121:8787/status,
Dashboard: http://10.148.0.121:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.121:37373,Workers: 0
Dashboard: http://10.148.0.121:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
#ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_demodulated_%s.zarr'%i) for i in [0,1000,2000,3000,3800]],dim = 'j')
ds = xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered.zarr')

In [7]:
ds.dims

Frozen({'face': 13, 'j': 4320, 'i': 4320, 'time': 8640})

In [30]:
ds.isel(face=4).SSU_rot_demodulated_real.mean('time').plot()

distributed.utils - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 681, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 346, in update
    self.root.title.text = title
AttributeError: 'str' object has no attribute 'text'
distributed.utils - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/utils.py", line 681, in log_errors
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 3417, in status_doc
    cluster_memory.update()
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/bokeh/core/property/v

KeyboardInterrupt: 

In [20]:
def merge(ds):
#    _ds = ds.chunk({'i':-1,'j':-1})
#    _ds = ds.chunk(dict(i=4320//4,j=4320//4)) #,j=4320//4
    _ds = ds
    _ds = ut._reset_chunk_encoding(_ds)    
    return _ds

In [5]:
def rechunked(ds):
    _ds = ds.chunk({'time':16,'i':4320//4,'j':4320//4})
    _ds = ut._reset_chunk_encoding(_ds)    
    return _ds

In [8]:
#Apply concat function on ds
face = 1
import os, dask
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_out, zarr = ut.custom_distribute(ds,#xr.merge([ds0,ds1]), 
                                        rechunked,
                                        overwrite=True,
                                        suffix="SSU_demodulated.zarr",
                                        tmp_dir=os.path.join(ut.root_data_dir, "filtered_itide"),
                                        append=True,
                                        face=face
#                                        j=200
                                       )

0it [03:36, ?it/s]


KilledWorker: ("('open_dataset-getitem-getitem-bc90690a53937fa1a9515c69b5509362', 0, 3, 35, 0)", <WorkerState 'tcp://10.148.1.21:54036', name: PBSCluster-4-5, status: closed, memory: 0, processing: 80>)

In [16]:
ds_out

,Array,Chunk
Bytes,15.25 TiB,1.24 GiB
Shape,"(13, 4320, 4320, 8640)","(1, 192, 100, 8640)"
Count,13157 Tasks,13156 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,15.25 TiB,1.24 GiB
Shape,"(13, 4320, 4320, 8640)","(1, 192, 100, 8640)"
Count,13157 Tasks,13156 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [9]:
#client.restart()
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
